In [1]:
import time

import numpy as np
from scipy.optimize import fmin_l_bfgs_b
from scipy.sparse import csr_matrix
from sklearn.base import BaseEstimator

from utils import *

In [2]:
def logloss_fun(x, y):
    return lambda theta: np.log(1 + np.exp(-y * x.dot(theta)))

In [3]:
def grad_logloss_fun(x, y):
    return lambda theta: 1 / (1 + np.exp(y * x.dot(theta)))

In [4]:
from sklearn.datasets import load_svmlight_file

def loading_dataset(filename):
    data = load_svmlight_file(filename)
    return data[0], data[1]

In [5]:
X, y = loading_dataset('Datasets/rcv1_train.binary')

In [6]:
y[y == -1] = 0

In [7]:
nnz_entries = np.unique(X.nonzero()[0])
X = X[nnz_entries]
y = y[nnz_entries]

In [8]:
class SqrtIterator():
    import numpy as np
    
    def __init__(self, gamma=1.0):
        self._t = 1
        self.gamma = gamma
    
    def __iter__(self):
        return self
    
    def __next__(self):
        sq = np.sqrt(self._t)
        self._t += 1
        return self.gamma * sq

In [9]:
sqrt_iter = SqrtIterator()
for _ in range(4):
    print(next(sqrt_iter))

1.0
1.41421356237
1.73205080757
2.0


In [35]:
class RDASolver(BaseEstimator):
    def __init__(self, lbda, gamma):
        """Implement the Regularized Dual Avering method [Xiao 2009] (cf. Algorithm 1)

        Args:
            w_dim (int): the dimension of the solution vector
            psi (func): penalization function (input: vector of shape (w_dim, ), output: float)
            aux (func): auxiliary function defined in the paper
            betas (iterator): non-negative non-decreasing series of float numbers
        """
        self.lbda = lbda
        self.gamma = gamma
        self._t = 1
        self._gBar = None
        self.betas = SqrtIterator(gamma)
        self.w = None
        self.losses = []
        self.log_likelihood = 0

    def iterate(self, X, y, g):
        # We compute the average of the past gradients
        self._gBar = (self._t - 1) / self._t * self._gBar + 1 / self._t * g

        # We can then define the objective function to minimize
        beta_t = next(self.betas)
        
        nnz_X = X.nonzero()[1]
        if nnz_X.size == 0:
            raise "Error at ligne %d " % (self._t + 1)
        
        # Update weight
        for i in nnz_X:
            if self._gBar[0, i] <= self.lbda:
                self.w[0, i] = 0
            else:
                sign = 1. if self._gBar[i] >= 0 else -1.
                w[0, i] = - beta_t / self.gamma * (self._gBar[0, i] - self.lbda * sign)
        
        # Compute probability and losses
        wtx = self.w.dot(X.T)
        p = sigmoid(wtx)
        self.log_likelihood += log_loss(y, p)
        self.losses.append(self.log_likelihood)
        
        self._t += 1
        return self.w, p

    def status(self):
        data = {"t": self._t, "w": self.w, "gBar": self._gBar}
        return data

    def fit(self, X, y):
        start_time = time.time()

        w_dim = X.shape[1]
        self.w = csr_matrix((1, w_dim))
        self._gBar = np.zeros((1, w_dim))
        y_proba = []

        for t in range(X.shape[0]):
            g = X[t] / (1 + np.exp(-y[t] * self.w.dot(X[t].T)[0, 0]))
            w, p = self.iterate(X[t], y[t], g)
            y_proba.append(p)
            if t % int(X.shape[0] / 10) == 0:
                print("Training Samples: %d |\t Loss: %s" % (t, self.log_likelihood))

        end_time = time.time()

    def predict(self, X):
        pass

In [36]:
rda = RDASolver(gamma=1.0, lbda=1.0)

In [37]:
N = 1000
np.random.seed(42)
i = np.random.choice(np.arange(X.shape[0]), N, replace=False)
X_sub = X[i]
y_sub = y[i]

In [38]:
rda.fit(X_sub, y_sub)

/Users/camille/.virtualenvs/ds/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Training Samples: 0 |	 Loss: [[ 0.69314718]]
Training Samples: 100 |	 Loss: [[ 70.00786524]]
Training Samples: 200 |	 Loss: [[ 139.32258329]]
Training Samples: 300 |	 Loss: [[ 208.63730135]]
Training Samples: 400 |	 Loss: [[ 277.9520194]]
Training Samples: 500 |	 Loss: [[ 347.26673746]]
Training Samples: 600 |	 Loss: [[ 416.58145552]]
Training Samples: 700 |	 Loss: [[ 485.89617357]]
Training Samples: 800 |	 Loss: [[ 555.21089163]]
Training Samples: 900 |	 Loss: [[ 624.52560968]]
